In [8]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from textblob import TextBlob
import ast

In [9]:
df_games = pd.read_csv('Datasets_limpio\\steam_games.csv', encoding='utf-8')
df_reviews = pd.read_csv('Datasets_limpio\\user_reviews.csv', encoding='utf-8')
df_items = pd.read_csv('Datasets_limpio\\user_items.csv', encoding='utf-8')
df_games['año_lanzamiento'].fillna(0, inplace=True)
df_games['año_lanzamiento'] = df_games['año_lanzamiento'].astype(int)

C:\Users\Manuel\AppData\Local\Temp\ipykernel_22892\2580637217.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_games['año_lanzamiento'].fillna(0, inplace=True)


In [10]:
df_reviews.head()

,user_id,reviews_posted,reviews_item_id,reviews_helpful,reviews_recommend,reviews_review
0,76561197970982479,"Posted November 5, 2011.",1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,js41637,"Posted June 24, 2014.",251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
2,evcentric,Posted February 3.,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...
3,doctr,"Posted October 14, 2013.",250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...
4,maplemage,"Posted April 15, 2014.",211420,35 of 43 people (81%) found this review helpful,True,Git gud


Para el analisis de sentimiento 

In [11]:
import pandas as pd

import nltk

from nltk.sentiment.vader import SentimentIntensityAnalyzer

from nltk.corpus import stopwords

from nltk.tokenize import word_tokenize

from nltk.stem import WordNetLemmatizer

In [12]:
# Descargar los recursos necesarios para VADER
nltk.download('vader_lexicon')

sid = SentimentIntensityAnalyzer()

def analizar_sentimiento(comentario):
    if pd.isnull(comentario) or comentario.lower() == 'sin comentario':
        return 1  # Neutral o sin review
    else:
        #Calcular la polaridad del sentimiento del comentario
        scores = sid.polarity_scores(comentario)
        if scores['compound'] < 0:
            return 0  # Malo
        elif scores['compound'] == 0:
            return 1  # Neutral
        elif scores['compound'] > 0:
            return 2  # Positivo
        



[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Manuel\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [13]:
df_reviews['sentiment_analysis'] = df_reviews['reviews_review'].apply(analizar_sentimiento)

In [14]:
df_reviews

,user_id,reviews_posted,reviews_item_id,reviews_helpful,reviews_recommend,reviews_review,sentiment_analysis
0,76561197970982479,"Posted November 5, 2011.",1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2
1,js41637,"Posted June 24, 2014.",251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,2
2,evcentric,Posted February 3.,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...,2
3,doctr,"Posted October 14, 2013.",250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...,2
4,maplemage,"Posted April 15, 2014.",211420,35 of 43 people (81%) found this review helpful,True,Git gud,1
...,...,...,...,...,...,...,...
59031,SKELETRONPRIMEISOP,"Posted August 15, 2014.",440,No ratings yet,True,TF2 is alot of fun and its really good but the...,2
59032,76561198141079508,"Posted August 2, 2014.",304930,No ratings yet,True,Fun game with friends,2
59033,ShadowYT100,"Posted July 31, 2015.",265630,No ratings yet,True,So Fun!! :D,2
59034,bestcustomurlevermade,"Posted December 20, 2015.",304050,No ratings yet,True,"This game is great. The only thing is,Why cant...",2


In [15]:
df_reviews = df_reviews.drop(columns=['reviews_review'])
df_reviews.columns

Index(['user_id', 'reviews_posted', 'reviews_item_id', 'reviews_helpful',
       'reviews_recommend', 'sentiment_analysis'],
      dtype='object')

In [16]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59036 entries, 0 to 59035
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             59036 non-null  object
 1   reviews_posted      59036 non-null  object
 2   reviews_item_id     59036 non-null  int64 
 3   reviews_helpful     59036 non-null  object
 4   reviews_recommend   59036 non-null  bool  
 5   sentiment_analysis  59036 non-null  int64 
dtypes: bool(1), int64(2), object(3)
memory usage: 2.3+ MB


Generamos los archivos para los endpoints con la informacion necesaria para los def

def developer( desarrollador : str ): Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora.

In [17]:

df_games_developer = df_games[['price', 'año_lanzamiento', 'developer', 'id']]
# Se renombra la columna 'id' relacionarla con el dataframe items
df_games_developer = df_games_developer.rename(columns={'id':'item_id'})
# se eliminan los duplicados
df_items_developer = df_games_developer.drop_duplicates()
df_items_developer

,price,año_lanzamiento,developer,item_id
0,4.99,2018,Kotoshiro,761140
5,0.00,2018,Secret Level SRL,643980
9,0.00,2017,Poolians.com,670290
14,0.99,2017,彼岸领域,767400
17,3.99,2018,Trickjump Games Ltd,772540
...,...,...,...,...
71534,1.99,2018,Bidoniera Games,745400
71538,1.99,2018,"Nikita ""Ghost_RUS""",773640
71542,4.99,2018,Sacada,733530
71545,1.99,2018,Laush Dmitriy Sergeevich,610660


In [18]:
archivo = 'Datasets_limpio\df_items_developer.csv'
df_items_developer.to_csv(archivo, index=False, encoding='utf-8')

In [19]:
archivo = 'arch_parquet\df_items_developer.parquet'
pq.write_table(pa.Table.from_pandas(df_items_developer), archivo)

def userdata( User_id : str ): Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.
Ejemplo de retorno: {"Usuario X" : us213ndjss09sdf, "Dinero gastado": 200 USD, "% de recomendación": 20%, "cantidad de items": 5}

In [20]:
df_items

,item_id,item_name,playtime_forever,steam_id,items_count,user_id
0,10,Counter-Strike,6,76561197970982479,277,76561197970982479
1,20,Team Fortress Classic,0,76561197970982479,277,76561197970982479
2,30,Day of Defeat,7,76561197970982479,277,76561197970982479
3,40,Deathmatch Classic,0,76561197970982479,277,76561197970982479
4,50,Half-Life: Opposing Force,0,76561197970982479,277,76561197970982479
...,...,...,...,...,...,...
5094100,346330,BrainBread 2,0,76561198329548331,7,76561198329548331
5094101,373330,All Is Dust,0,76561198329548331,7,76561198329548331
5094102,388490,One Way To Die: Steam Edition,3,76561198329548331,7,76561198329548331
5094103,521570,You Have 10 Seconds 2,4,76561198329548331,7,76561198329548331


In [21]:
# Estraigo las columnas 'items_count', 'user_id' e 'item_id'
df_gastos_items = df_items[['items_count', 'user_id', 'item_id']]
df_gastos_items

,items_count,user_id,item_id
0,277,76561197970982479,10
1,277,76561197970982479,20
2,277,76561197970982479,30
3,277,76561197970982479,40
4,277,76561197970982479,50
...,...,...,...
5094100,7,76561198329548331,346330
5094101,7,76561198329548331,373330
5094102,7,76561198329548331,388490
5094103,7,76561198329548331,521570


In [22]:
#nuevo df con el precio de los juegos
precio_juegos = df_games[['price', 'id']]
# Elimina los duplicados
precio_juegos = precio_juegos.drop_duplicates(subset='id', keep='first')
# Se renombra 'id' por 'item_id' para unir mas adelante
precio_juegos = precio_juegos.rename(columns={'id':'item_id'})
precio_juegos

,price,item_id
0,4.99,761140
5,0.00,643980
9,0.00,670290
14,0.99,767400
17,3.99,772540
...,...,...
71534,1.99,745400
71538,1.99,773640
71542,4.99,733530
71545,1.99,610660


In [23]:
#uno los dataframes
df_gastos_items = df_gastos_items.merge(precio_juegos, on='item_id', how='left')
df_gastos_items

,items_count,user_id,item_id,price
0,277,76561197970982479,10,9.99
1,277,76561197970982479,20,4.99
2,277,76561197970982479,30,4.99
3,277,76561197970982479,40,4.99
4,277,76561197970982479,50,4.99
...,...,...,...,...
5094100,7,76561198329548331,346330,0.00
5094101,7,76561198329548331,373330,NaN
5094102,7,76561198329548331,388490,0.00
5094103,7,76561198329548331,521570,0.00


In [24]:
df_gastos_items[df_gastos_items['price'].isnull()]

,items_count,user_id,item_id,price
13,277,76561197970982479,34440,NaN
26,277,76561197970982479,9340,NaN
27,277,76561197970982479,228200,NaN
37,277,76561197970982479,17340,NaN
51,277,76561197970982479,23120,NaN
...,...,...,...,...
5094089,177,76561198326700687,508380,NaN
5094091,177,76561198326700687,510660,NaN
5094095,177,76561198326700687,519170,NaN
5094101,7,76561198329548331,373330,NaN


In [25]:
df_temporal = df_gastos_items['price'].fillna(0.0)
# borro y agrego la nueva columna
df_gastos_items = pd.concat([df_gastos_items.drop('price', axis=1), df_temporal], axis=1)
df_gastos_items

,items_count,user_id,item_id,price
0,277,76561197970982479,10,9.99
1,277,76561197970982479,20,4.99
2,277,76561197970982479,30,4.99
3,277,76561197970982479,40,4.99
4,277,76561197970982479,50,4.99
...,...,...,...,...
5094100,7,76561198329548331,346330,0.00
5094101,7,76561198329548331,373330,0.00
5094102,7,76561198329548331,388490,0.00
5094103,7,76561198329548331,521570,0.00


In [26]:
# Agrupor por usuario para hallar la cantidad gastada por usuario
df_gastos_items = df_gastos_items.drop('item_id', axis=1)
df_gastos_usuario = df_gastos_items.groupby('user_id')['price'].sum().reset_index()
df_gastos_usuario

,user_id,price
0,--000--,397.78
1,--ace--,166.82
2,--ionex--,99.93
3,-2SV-vuLB-Kg,427.50
4,-404PageNotFound-,1509.32
...,...,...
70907,zzonci,19.98
70908,zzoptimuszz,64.98
70909,zzydrax,99.94
70910,zzyfo,828.51


In [27]:
df_count_items = df_gastos_items[['items_count', 'user_id']]
# se eliminan los duplicados
df_count_items = df_count_items.drop_duplicates(subset='user_id', keep='first')
df_gastos_items = df_count_items.merge(df_gastos_usuario, on='user_id', how='right')
df_gastos_items

,items_count,user_id,price
0,58,--000--,397.78
1,44,--ace--,166.82
2,23,--ionex--,99.93
3,68,-2SV-vuLB-Kg,427.50
4,149,-404PageNotFound-,1509.32
...,...,...,...
70907,5,zzonci,19.98
70908,61,zzoptimuszz,64.98
70909,13,zzydrax,99.94
70910,84,zzyfo,828.51


In [28]:
archivo = 'Datasets_limpio\df_gastos_items.csv'
df_gastos_items.to_csv(archivo, index=False, encoding='utf-8')

In [29]:
#lo guardo como parquet
archivo = 'arch_parquet\gastos_items.parquet'

pq.write_table(pa.Table.from_pandas(df_gastos_items), archivo)

In [30]:
df_games

,publisher,genres,app_name,title,price,early_access,id,developer,año_lanzamiento
0,Kotoshiro,Action,Lost Summoner Kitty,Lost Summoner Kitty,4.99,False,761140,Kotoshiro,2018
1,Kotoshiro,Casual,Lost Summoner Kitty,Lost Summoner Kitty,4.99,False,761140,Kotoshiro,2018
2,Kotoshiro,Indie,Lost Summoner Kitty,Lost Summoner Kitty,4.99,False,761140,Kotoshiro,2018
3,Kotoshiro,Simulation,Lost Summoner Kitty,Lost Summoner Kitty,4.99,False,761140,Kotoshiro,2018
4,Kotoshiro,Strategy,Lost Summoner Kitty,Lost Summoner Kitty,4.99,False,761140,Kotoshiro,2018
...,...,...,...,...,...,...,...,...,...
71545,Laush Studio,Indie,Russian Roads,Russian Roads,1.99,False,610660,Laush Dmitriy Sergeevich,2018
71546,Laush Studio,Racing,Russian Roads,Russian Roads,1.99,False,610660,Laush Dmitriy Sergeevich,2018
71547,Laush Studio,Simulation,Russian Roads,Russian Roads,1.99,False,610660,Laush Dmitriy Sergeevich,2018
71548,SIXNAILS,Casual,EXIT 2 - Directions,EXIT 2 - Directions,4.99,False,658870,"xropi,stev3ns",2017


In [31]:
df_reviews

,user_id,reviews_posted,reviews_item_id,reviews_helpful,reviews_recommend,sentiment_analysis
0,76561197970982479,"Posted November 5, 2011.",1250,No ratings yet,True,2
1,js41637,"Posted June 24, 2014.",251610,15 of 20 people (75%) found this review helpful,True,2
2,evcentric,Posted February 3.,248820,No ratings yet,True,2
3,doctr,"Posted October 14, 2013.",250320,2 of 2 people (100%) found this review helpful,True,2
4,maplemage,"Posted April 15, 2014.",211420,35 of 43 people (81%) found this review helpful,True,1
...,...,...,...,...,...,...
59031,SKELETRONPRIMEISOP,"Posted August 15, 2014.",440,No ratings yet,True,2
59032,76561198141079508,"Posted August 2, 2014.",304930,No ratings yet,True,2
59033,ShadowYT100,"Posted July 31, 2015.",265630,No ratings yet,True,2
59034,bestcustomurlevermade,"Posted December 20, 2015.",304050,No ratings yet,True,2


In [32]:
anio_lanzamiento = df_games[['id', 'año_lanzamiento','developer','title']]
# Se renombra la columna 'id' para unirla con el dataframe anterior
anio_lanzamiento = anio_lanzamiento.rename(columns={'id':'reviews_item_id'})
# se eliminan los duplicados
anio_lanzamiento = anio_lanzamiento.drop_duplicates()


df_reviews = df_reviews.merge(anio_lanzamiento, on='reviews_item_id')
df_reviews

,user_id,reviews_posted,reviews_item_id,reviews_helpful,reviews_recommend,sentiment_analysis,año_lanzamiento,developer,title
0,76561197970982479,"Posted November 5, 2011.",1250,No ratings yet,True,2,2009,Tripwire Interactive,Killing Floor
1,evcentric,Posted February 3.,248820,No ratings yet,True,2,2013,"Hopoo Games, LLC",Risk of Rain
2,doctr,"Posted October 14, 2013.",250320,2 of 2 people (100%) found this review helpful,True,2,2013,Telltale Games,The Wolf Among Us
3,maplemage,"Posted April 15, 2014.",211420,35 of 43 people (81%) found this review helpful,True,1,2012,FromSoftware,DARK SOULS™: Prepare To Die™ Edition
4,Wackky,"Posted May 5, 2014.",249130,7 of 8 people (88%) found this review helpful,True,1,2013,Traveller's Tales,LEGO® Marvel™ Super Heroes
...,...,...,...,...,...,...,...,...,...
50213,76561198138691719,"Posted December 30, 2015.",332310,No ratings yet,True,2,2017,"TT Games,Traveller's Tales",LEGO® Worlds
50214,SKELETRONPRIMEISOP,"Posted August 15, 2014.",440,No ratings yet,True,2,2007,Valve,Team Fortress 2
50215,76561198141079508,"Posted August 2, 2014.",304930,No ratings yet,True,2,2017,Smartly Dressed Games,Unturned
50216,ShadowYT100,"Posted July 31, 2015.",265630,No ratings yet,True,2,2014,Fistful of Frags Team,Fistful of Frags


In [33]:
archivo = 'Datasets_limpio\df_reviews.csv'
df_reviews.to_csv(archivo, index=False, encoding='utf-8')

archivo = 'arch_parquet\df_reviews.parquet'
pq.write_table(pa.Table.from_pandas(df_reviews), archivo)

def UserForGenre( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento.

In [34]:
playtime_forever = df_items[['playtime_forever', 'user_id', 'item_id']]
playtime_forever

,playtime_forever,user_id,item_id
0,6,76561197970982479,10
1,0,76561197970982479,20
2,7,76561197970982479,30
3,0,76561197970982479,40
4,0,76561197970982479,50
...,...,...,...
5094100,0,76561198329548331,346330
5094101,0,76561198329548331,373330
5094102,3,76561198329548331,388490
5094103,4,76561198329548331,521570


In [35]:
genre_item = df_games[['genres','año_lanzamiento','id']]
# Modifico la columna 'id' para unirla con el dataframe anterior
genre_item = genre_item.rename(columns={'id':'item_id'})
genre_item

,genres,año_lanzamiento,item_id
0,Action,2018,761140
1,Casual,2018,761140
2,Indie,2018,761140
3,Simulation,2018,761140
4,Strategy,2018,761140
...,...,...,...
71545,Indie,2018,610660
71546,Racing,2018,610660
71547,Simulation,2018,610660
71548,Casual,2017,658870


In [36]:
#Uno las dos tablas
df_playtime_genre_aux = playtime_forever.merge(genre_item, on='item_id')
df_playtime_genre= df_playtime_genre_aux.drop(columns=['item_id'])


In [37]:
df_playtime_genre

,playtime_forever,user_id,genres,año_lanzamiento
0,6,76561197970982479,Action,2000
1,0,76561197970982479,Action,1999
2,7,76561197970982479,Action,2003
3,0,76561197970982479,Action,2001
4,0,76561197970982479,Action,1999
...,...,...,...,...
9877299,3,76561198329548331,Adventure,2015
9877300,3,76561198329548331,Free to Play,2015
9877301,4,76561198329548331,Casual,2016
9877302,4,76561198329548331,Free to Play,2016


In [38]:
# Considero la columna 'playtime_forever' a 'hours'
df_playtime_genre['playtime_horas'] = df_playtime_genre['playtime_forever']/60
# Se borra la columna en minutos
df_playtime_genre = df_playtime_genre.drop('playtime_forever', axis=1)
df_playtime_genre

,user_id,genres,año_lanzamiento,playtime_horas
0,76561197970982479,Action,2000,0.100000
1,76561197970982479,Action,1999,0.000000
2,76561197970982479,Action,2003,0.116667
3,76561197970982479,Action,2001,0.000000
4,76561197970982479,Action,1999,0.000000
...,...,...,...,...
9877299,76561198329548331,Adventure,2015,0.050000
9877300,76561198329548331,Free to Play,2015,0.050000
9877301,76561198329548331,Casual,2016,0.066667
9877302,76561198329548331,Free to Play,2016,0.066667


In [39]:
df_playtime_genre['playtime_horas'] = df_playtime_genre['playtime_horas'].astype('int32')

In [40]:
df_playtime_genre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9877304 entries, 0 to 9877303
Data columns (total 4 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   user_id          object
 1   genres           object
 2   año_lanzamiento  int32 
 3   playtime_horas   int32 
dtypes: int32(2), object(2)
memory usage: 226.1+ MB


In [41]:
# Agrupar por user_id, año_lanzamiento y sumar las horas jugadas
df_playtime_genre2 = df_playtime_genre.groupby(['user_id','genres','año_lanzamiento'])['playtime_horas'].sum().reset_index()


In [42]:
archivo = 'arch_parquet\df_anio_horas.parquet'
pq.write_table(pa.Table.from_pandas(df_playtime_genre2), archivo)

: 